In [33]:
import csv
import requests
import xml.etree.ElementTree as ET 
import pandas as pd 

In [2]:
tree = ET.parse('ioofservicT1-Metadata.xml')

In [4]:
root = tree.getroot()

### Get all table fields info

In [61]:
fields = []

In [62]:
for index, child in enumerate(root[0][1]):
    for item in child:
        if 'EntityType' in child.tag:
            field = item.attrib
            field['Table'] = child.attrib['Name']
            fields.append(field)
        else:
            continue

In [64]:
df_fields = pd.DataFrame.from_records(fields)

### Get all table level info

In [179]:
tables = []

In [180]:
for index, child in enumerate(root[0][0][0]):
    if 'EntitySet' in child.tag:
        table = child.attrib
        for grand_child in child:
            try:
                table['Description'] = grand_child[1].text
            except:
                table['Description'] = ''
        tables.append(table)

In [182]:
df_tables = pd.DataFrame.from_records(tables)

In [184]:
df_tables.shape

(620, 8)

### Extract relationship

In [185]:
associations = []

In [200]:
for index, child in enumerate(root[0][0][0]):
    # print(child.tag)
    for item in child:
        if 'AssociationSet' in child.tag:
            association = child.attrib
            association['ToTable'] = child[1].attrib['EntitySet']
        else:
            continue
    associations.append(association)

In [203]:
df_associations = pd.DataFrame.from_records(associations).drop_duplicates()

In [205]:
df_fields.drop_duplicates()

,Table,Name,Type,Nullable,{http://www.successfactors.com/edm/sap}required,{http://www.successfactors.com/edm/sap}creatable,{http://www.successfactors.com/edm/sap}updatable,{http://www.successfactors.com/edm/sap}upsertable,{http://www.successfactors.com/edm/sap}visible,{http://www.successfactors.com/edm/sap}sortable,...,FromRole,ToRole,{http://www.successfactors.com/edm/sap}field-control,{http://www.successfactors.com/edm/sap}picklist,{http://www.successfactors.com/edm/sap}inlineRequired,Precision,Scale,DefaultValue,{http://www.successfactors.com/edm/sap}sensitive-personal-data,{http://www.successfactors.com/edm/sap}elm-strength
0,Entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Entity,deletable,Edm.Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Entity,deletablePath,Edm.String,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Entity,effectiveDated,Edm.Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Entity,insertable,Edm.Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,NominationTarget,nominationId,Edm.Int64,false,true,false,false,false,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13196,NominationTarget,nominationType,Edm.Int32,false,true,false,false,false,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13197,NominationTarget,positionNav,NaN,NaN,false,false,false,false,true,true,...,NominationTarget,Position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13198,NominationTarget,successorNav,NaN,NaN,false,false,false,false,true,true,...,NominationTarget,Successor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Post extract processing

##### 1. df_fiedls table

In [257]:
df_fields['Type'] = df_fields['Type'].str.replace('Edm.', '')

In [246]:
cols = df_fields.columns.to_list()

In [248]:
for idx, col in enumerate(cols):
    if '{http://www.successfactors.com/edm/sap}' in col:
        col = col.replace('{http://www.successfactors.com/edm/sap}', '')
        cols[idx] = col
    else:
        continue

In [250]:
df_fields.columns = cols

In [258]:
df_fields.head(2)

,Table,Name,Type,Nullable,required,creatable,updatable,upsertable,visible,sortable,...,FromRole,ToRole,field-control,picklist,inlineRequired,Precision,Scale,DefaultValue,sensitive-personal-data,elm-strength
0,Entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Entity,deletable,Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 2. df_tables

In [259]:
df_tables.columns

Index(['Name', 'EntityType', '{http://www.successfactors.com/edm/sap}label',
       '{http://www.successfactors.com/edm/sap}creatable',
       '{http://www.successfactors.com/edm/sap}updatable',
       '{http://www.successfactors.com/edm/sap}upsertable',
       '{http://www.successfactors.com/edm/sap}deletable', 'Description'],
      dtype='object')

In [262]:
cols = df_tables.columns.to_list()

In [263]:
for idx, col in enumerate(cols):
    if '{http://www.successfactors.com/edm/sap}' in col:
        col = col.replace('{http://www.successfactors.com/edm/sap}', '')
        cols[idx] = col
    else:
        continue

In [270]:
df_tables.columns = cols

In [271]:
cols

['Name',
 'EntityType',
 'label',
 'creatable',
 'updatable',
 'upsertable',
 'deletable',
 'Description']

In [273]:
df_tables = df_tables[['Name', 'EntityType', 'label', 'Description']]

##### 3. df_associations

In [274]:
cols = df_associations.columns.to_list()

In [275]:
for idx, col in enumerate(cols):
    if '{http://www.successfactors.com/edm/sap}' in col:
        col = col.replace('{http://www.successfactors.com/edm/sap}', '')
        cols[idx] = col
    else:
        continue

In [276]:
df_associations.columns = cols

In [277]:
df_associations

,Name,Association,ToTable
0,mdfSystemRecordStatusNav_of_EmpCostDistribution,SFOData.mdfSystemRecordStatusNav_of_EmpCostDis...,MDFEnumValue
620,mdfSystemStatusNav_of_TimeAccountDetail,SFOData.mdfSystemStatusNav_of_TimeAccountDetail,MDFEnumValue
621,Background_Languages_variant_picklist,SFOData.Background_Languages_variant_picklist,PicklistOption
622,periodModelNav_of_WorkSchedule,SFOData.periodModelNav_of_WorkSchedule,WorkSchedule
623,legalEntityNav_of_PayrollDataMaintenanceTask,SFOData.legalEntityNav_of_PayrollDataMaintenan...,FOCompany
...,...,...,...
4110,PersonKey_UserAccount,SFOData.PersonKey_UserAccount,UserAccount
4111,legalEntityTypeNav_of_FOLegalEntityLocalUSA,SFOData.legalEntityTypeNav_of_FOLegalEntityLoc...,PickListValueV2
4112,mdfSystemStatusNav_of_EmployeeTimePOL,SFOData.mdfSystemStatusNav_of_EmployeeTimePOL,MDFEnumValue
4113,mdfSystemStatusNav_of_ShiftClassification,SFOData.mdfSystemStatusNav_of_ShiftClassification,MDFEnumValue


### Merge tables